In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Convolution2D, Flatten, Dropout,GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import models,optimizers
from keras import layers
from tensorflow import keras
from keras import metrics
from keras.applications.inception_resnet_v2 import preprocess_input
from attention import AttentionLayer
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
import os
import pandas as pd
import matplotlib.pyplot as plt
#import keras
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
def plotResults(history):
        
    # Accuracy
    plt.plot(history.history["accuracy"])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    #plt.savefig('images/accuracy.png')
    plt.show()
    plt.close()
    
    # Loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    #plt.savefig('images/loss.png')
    plt.show()
    plt.close()

In [ ]:
datadir = "./data/akac_gen.txt"

In [ ]:
dataset = pd.read_csv(datadir, delimiter ="\t", header = 0)
print(list(dataset.columns))
dataset = dataset.sample(frac=1,random_state=777)  # row 전체 shuffle

SNPs = ['rs2235375','rs1044516','rs11204737','rs1044516','rs7715100','rs3917192','rs2235371']
snps = ['rs906830', 'rs1537514', 'rs4252328', 'rs2235371', 'rs481931', 'rs3771485', 'rs4252328', 'rs9651118', 'rs11796677', 'rs2295221']

dataset.head()
del_list = ["famid", "pid", "fatid", "motid", "sex","rs12089548","rs11466414"]
for i in del_list:
    del dataset[i]

y_train = pd.to_numeric(dataset['affected']) -1
del dataset['affected']




In [ ]:
dataset.head()

In [ ]:
#from keras.utils import to_categorical
#y_train_np = to_categorical(y_train_np)
#print(y_train_np)

In [ ]:
model = keras.models.load_model("./Ensemble/10snp2_ann.h5")
#model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])
#model.evaluate(x_train_np, y_train_np, batch_size=200, verbose=2)
#model.evaluate(x_test_np, y_test_np, batch_size=100, verbose=2)


In [ ]:
#model1 = keras.models.load_model("./Ensemble/10snp1_ann-70-0.63.h5")
model1 = keras.models.load_model("./Ensemble/10snp1_ann_new.h5")
model2 = keras.models.load_model("./Ensemble/10snp2_ann-87-0.63.h5")
model3 = keras.models.load_model("./Ensemble/10snp3_ann-70-0.65.h5")
model4 = keras.models.load_model("./Ensemble/10snp4_ann-60-0.60.h5")
model5 = keras.models.load_model("./Ensemble/10snp5_ann-99-0.68.h5")
model6 = keras.models.load_model("./Ensemble/10snp6_ann-91-0.68.h5")

In [ ]:
models = [model1,model2,model3,model4,model5,model6]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#model.load_weights("./model/16snp_ann-196-0.63.h5")
#test_loss, test_acc = model1.evaluate(x_train_np, y_train_np, batch_size=200, verbose=2)
predict_test =np.zeros((62,1))
predict_train =np.zeros((200,1))
raw_output_test=[]
raw_output_train=[]
num_train =200
class CustomStopper(keras.callbacks.EarlyStopping):
    def __init__(self, monitor='val_accuracy',patience=50, verbose=1,restore_best_weights=True, mode='auto', start_epoch=20):# add argument for starting epoch
        super(CustomStopper, self).__init__(monitor=monitor,patience=patience,mode=mode,verbose=verbose,restore_best_weights=restore_best_weights)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)
SNPs = [['rs7078160','rs3753582','rs2235373','rs1429591','rs1801133','rs906830','rs2013162','rs1044516','rs3917192','rs7103685'],
           ['rs2235373','rs2013162','rs1044516','rs11204737','rs595918','rs16873348','rs3753582','rs2284791','rs3917192','rs7715100'],
           ['rs3917192', 'rs16873348', 'rs2013162', 'rs1044516', 'rs2284791', 'rs9651118', 'rs1429591', 'rs7103685', 'rs1801133', 'rs17317411'],
           ['rs3917192', 'rs16873348', 'rs481931', 'rs3771485', 'rs3755377', 'rs9651118', 'rs595918', 'rs7103685', 'rs1801133', 'rs2013162'],
           ['rs906830', 'rs1537514', 'rs4252328', 'rs2235371', 'rs481931', 'rs3771485', 'rs2013162', 'rs9651118', 'rs11796677', 'rs2295221'],
       ['rs3917192', 'rs16873348', 'rs2013162', 'rs1044516', 'rs7078160', 'rs9651118', 'rs1429591', 'rs7103685', 'rs1801133', 'rs595918']]
print(SNPs[0])

#raw_output_test.append(y_test_np)
#raw_output_train.append(y_train_np)

for i,j in zip(models,range(len(models))):
    X_train = dataset[SNPs[j]]
    x_train_np = np.array(X_train)[:num_train].astype("float")
    x_test_np = np.array(X_train)[num_train:].astype("float")
    y_train_np = np.array(y_train)[:num_train].astype("float")
    y_test_np = np.array(y_train)[num_train:].astype("float")
    
    i.evaluate(x_train_np, y_train_np, batch_size=200, verbose=2)
    i.evaluate(x_test_np, y_test_np, batch_size=100, verbose=2)
    predict_test+=i.predict(x_test_np, batch_size=100)
    predict_train+=i.predict(x_train_np, batch_size=100)
    raw_output_test.append(i.predict(x_test_np, batch_size=100))
    raw_output_train.append(i.predict(x_train_np, batch_size=100))
    
threshold = 5.5
print('test auc')
print(roc_auc_score(y_test_np, predict_test))
print('train auc')
print(accuracy_score(y_train_np, np.round(predict_train/threshold,0)))
print('test acc')
print(accuracy_score(y_test_np, np.round(predict_test/threshold,0)))
tn, fp, fn, tp = confusion_matrix(y_test_np, np.round(predict_test/threshold,0)).ravel()
specificity = tn/(tn+fp)
sensitivity = tp/(tp+fn)
print('specificity')
print(specificity)
print('sensitivity')
print(sensitivity)

#['rs2013162', 'rs595918', 'rs16873348', 'rs2235373', 'rs2284791', 'rs2073247', 'rs1044516', 'rs11204737', 'rs3917192', 'rs7715100']

##### print(raw_output_test[0].shape)
arr = np.array(raw_output_test).squeeze()
print(np.expand_dims(y_test_np,axis=0).shape)
arr = np.append(arr,y_test_np.reshape(1,62), axis=0)
print(arr.shape)
df_out = pd.DataFrame(np.transpose(arr))
df_out.to_csv('output.csv',index=False)

In [ ]:
from sklearn.metrics import roc_curve, auc, f1_score
print('Classification Report')
print(classification_report(y_test_np, np.round(predict_test/6,0), target_names=['0','1'], digits=4))
print(classification_report(y_test_np, np.round(raw_output_test[1],0), target_names=['0','1'], digits=4))

In [ ]:
def plot_confusion_matrix2(name,actual, predicted, classes, normalize=True, title='Confusion Matrix', cmap=plt.cm.viridis, dpi=150):
    #conf_matrix = pd.crosstab(actual, predicted)  # confusion_matrix(actual, predicted)
    from sklearn.utils.multiclass import unique_labels
    conf_matrix = confusion_matrix(actual, predicted)
    classes = classes
    if normalize:
        conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] *100
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(8, 6))
    plt.imshow(conf_matrix, interpolation='nearest', cmap=cmap)  # , cmap=plt.cm.Greens
    #plt.title(title, size=12)
    plt.colorbar(fraction=0.05, pad=0.05)
    tick_marks = np.arange(len(classes))
    #tick_marks = np.arange(len(classes))
    #plt.xticks(tick_marks, classes, rotation=0)
    plt.xticks(np.arange(len(classes)), classes)
    plt.yticks(np.arange(len(classes)), classes)
    plt.ylim( len(classes) - 0.5,-0.5)
    #plt.yticks(np.arange(8), ('C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'))
    #plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = conf_matrix.max() / 2.
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            plt.text(j, i, format(conf_matrix[i, j], fmt),
                     ha="center", va="center", color="white" if conf_matrix[i, j] > thresh else "black")  #horizontalalignment
    plt.tight_layout()
    plt.savefig(name, dpi=dpi)
    
    return plt
   
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.grid(False)
    #plt.tight_layout()


In [ ]:
dpi = 150
print(confusion_matrix(y_train_np, np.round(predict_train/5.5,0)))
plot_confusion_matrix2(name ="./confusion_10.png".format(dpi),actual = y_test_np, predicted =np.round(predict_test/6,0), classes =['0','1'], cmap = 'Blues', dpi=dpi )

In [ ]:
from sklearn.metrics import roc_curve, auc, f1_score

nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test_np, predict_test)
t, o, _ = roc_curve(y_test_np, raw_output_test[4])
print(roc_auc_score(y_test_np, raw_output_test[4]))
print(nn_fpr_keras)
print(nn_tpr_keras)
print(t)
print(o)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot([0, 1], [0, 1], "k--", lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("1 - Specificity")
plt.ylabel("Sensitivity")
plt.gca().set_aspect('equal', adjustable='box')

plt.title('10 SNPs AUC')
plt.plot([0,0.0588,0.1176,0.2941,0.4705,0.5882,1], [0,0.3333,0.4222,0.8888,0.9333,0.9777,1], label='GANNET(0.882)',color='r')
plt.plot([0,0.1176,0.1764,0.2941,0.4117,0.7647,0.9411,1], [0,0.1555,0.3111,0.8444,0.8444,0.9555,0.9777,1], label='ANN(0.714)',color='C1')
plt.plot([0,0.0839,0.2027,0.3986,0.5104,0.8041,1], [0,0.2100,0.4201,0.6302,0.7310,0.9495,1], label='PRS(0.657)', color = 'C8')
plt.plot([0.,0.40740741,1.], [0.,0.57142857,1.], label='RF(0.597)',color='C2')
plt.plot([0.,0.25925926,1.], [0.,0.62857143,1.], label='SVM(0.685)',color='C4')
plt.plot([0.,0.2962963,1.], [0.,0.54285714,1.], label='XGB(0.623)',color='C0')
plt.plot([0.,0.3333,1.], [0.,0.57,1.], label='LR(0.619)', color = 'C5')
plt.plot([0.,0.4074,1.], [0.,0.5429,1.], label='LGBM(0.568)', color = 'C6')
plt.plot([0.,0.2962,1.], [0.,0.5714,1.], label='ADA(0.638)', color = 'C9')


plt.legend(loc='best',frameon=False, prop={'size': 8})
plt.savefig("./figure/10snp_roc.png",dpi=300)

In [ ]:
plt.plot([0, 1], [0, 1], "k--", lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("1 - Specificity")
plt.ylabel("Sensitivity")
plt.gca().set_aspect('equal', adjustable='box')

plt.title('10 SNPs AUC')
plt.plot([0,0.0588,0.1176,0.2941,0.4705,0.5882,1], [0,0.3333,0.4222,0.8888,0.9333,0.9777,1], label='GANNET(0.882)',color='r')
plt.plot([0,0.1176,0.1764,0.2941,0.4117,0.7647,0.9411,1], [0,0.1555,0.3111,0.8444,0.8444,0.9555,0.9777,1], label='ANN(0.714)',color='C1')
plt.plot([0,0.0839,0.2027,0.3986,0.5104,0.8041,1], [0,0.2100,0.4201,0.6302,0.7310,0.9495,1], label='PRS(0.657)', color = 'C8')
plt.plot([0.,0.40740741,1.], [0.,0.57142857,1.], label='RF(0.597)',color='C2')
plt.plot([0.,0.25925926,1.], [0.,0.62857143,1.], label='SVM(0.677)',color='C4')
plt.plot([0.,0.2962963,1.], [0.,0.54285714,1.], label='XGB(0.613)',color='C0')


plt.legend(loc='best',frameon=False, prop={'size': 9})
plt.savefig("./figure/10snp_roc.png",dpi=300)

In [ ]:
def softmax(arr):
    m = np.argmax(arr)
    arr = arr - m
    arr = np.exp(arr)
    return arr / np.sum(arr)

In [ ]:
num = 15
outputs = model.predict(x_test_np[:])
print(y_test_np[num])
#print(outputs[0])
model_outputs = np.array(outputs[0])
attention_outputs = outputs[1]

In [ ]:
index = 0
sav_ind = []
for t, k in zip(y_test_np, np.squeeze(np.round(model_outputs))):
    if(t == k):
        sav_ind.append(index)
    index+=1
print(sav_ind)
print(list(dataset.columns))

att = np.zeros(shape=(92,1))
index =0
for i in attention_outputs:
    if index in sav_ind:
        att+=np.expand_dims(np.array(attention_outputs[4][:]),axis=1)
    #att+=np.expand_dims(np.array(attention_outputs[4][:]),axis=1)
    index+=1

In [ ]:
#print(att)
#print(np.where(np.squeeze(att/np.amax(att))==1))
sorted_idx = sorted(range(len(att)),key=lambda x:att[x])
att = [att[i] for i in sorted_idx ][::-1]
snps_list = [list(dataset.columns)[i] for i in sorted_idx ][::-1]
best_snp = np.where(np.squeeze(att/np.amax(att))==1)
for i in best_snp[0]:
    print(snps_list[i])
print(snps_list)

In [ ]:
plt.bar(np.arange(len(np.squeeze(att/np.amax(att)))),np.squeeze(att/np.amax(att)))
plt.show()